# Data Acquisition and Exploration

This notebook was created as part of a workshop on *Reproducible Research in Python*. 

- You can access the entire workshop materials at: [Reproducible Research in Python](https://github.com/mickaeltemporao/reproducible-research-in-python).

**Learning Objective:** 
- Get familiar with common data exploration libraries
- Learn to acquire and clean data
- Learn to explore and visualize 



## Acquiring Data
With some Python basics we will start combining existing packages to acquire, and explore data.


In [0]:
# Install requirred libraries
!pip install wikipedia

In [0]:
# Load the required libraries
import pandas as pd
import wikipedia as wp
import matplotlib
matplotlib.rcParams['figure.figsize'] = [10, 5]


In [0]:
# Identify Wikipedia Page and acquire the date
page_title = "Opinion polling for the 2019 Canadian federal election"


In [0]:
#Get the html source
html = wp.page(page_title).html().encode("UTF-8")


In [0]:
# Extract tables and convert the html tables into pd.DataFrame()
df = pd.read_html(html)[0].iloc[2:,:]


## Cleaning Data


In [0]:
# Inspect the data
df.head()


In [0]:
# We notice that there seems to be a double header
df.columns


In [0]:
# What is the type of columns
type(df.columns)


In [0]:
# Let's use a loop to extract and edit each element of the MultiIndex dataframe
columnn_names = []
for c in df.columns:
    tmp = c[0].lower()
    columnn_names.append(tmp.replace(" ", "_"))

columnn_names


In [0]:
# Let's use regular expressions in a list comprehension this time
import re
regex = "[a-z]+"
columnn_names = ["_".join(re.findall(regex, i)) for i in columnn_names]


In [0]:
# Let's edit the columns of our dataset
df.columns = columnn_names
df.head()


In [0]:
# Let's further rename those columns
names_dict = {
    "polling_firm": "source",
    "last_dateof_polling": "date",
    "samplesize": "sample_size",
    "marginof_error": "error",
    "cons": "cpc",
    "liberal": "lpc",
    "green": "gpc",
    "polling_method": "method",
}

type(names_dict)


In [0]:
# Pass the new dictionary as an argument to the .rename method
df.rename(columns=names_dict, inplace=True)
df.head()


In [0]:
# Let's check the data types
df.dtypes


In [0]:
# The date field needs to be converted
df[['date']] = pd.to_datetime(df.date)
df.head()


In [0]:
# We should also only keep the numeric values for the margins of error
regex = "(\d+\.*\d*)"
df.error = df.error.str.extract(regex)


In [0]:
# Let's look again at our dataset
df.head()


In [0]:
# What if we look at a random subsample
df.sample(5)


In [0]:
# Let's clean the sample
regex = r"\(.*\)"
df.sample_size = df.sample_size.str.replace(regex, "")
df.sample_size = df.sample_size.str.replace(" |,", "")


In [0]:
# How does the data look now?
df.sample(5)


In [0]:
# What about the data types?
df.info()


In [0]:
# Which of these variables are still objects?
df.select_dtypes(include='object')


In [0]:
# Let's use a dictionary to recode the data types
convert_dict = {
    'error': float,
    'sample_size': int,
    'lead': float
}

df = df.astype(convert_dict)


In [0]:
# Let's look once again at our data
df.sample(5)


In [0]:
# What are the remaining objects?
df.select_dtypes(include='object')


In [0]:
# Keep only necessary variables by creating a variable filter
to_keep = [
    'source',
    'date',
    'lpc',
    'cpc',
    'ndp',
    'bq',
    'gpc',
    'method'
]

df = df[to_keep]


## Data IO


In [0]:
# Save the cleaned dataframe to a file
file_name = "national_polls_2019.csv"
df.to_csv(file_name, index=False)
print(df)

df.dtypes


In [0]:
# Read the data back-in from the recorded csv file.

# More info on read_csv
# https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_csv.html
df = pd.read_csv("national_polls_2019.csv", parse_dates=['date'])
df.dtypes


## Data Exploration and Visualization


In [0]:
# Let's convert this into a time-series dataframe
df.set_index('date', inplace=True)


In [0]:
# Time-series data should be stored in descending order
df = df.sort_values(by=['date', 'source'])


In [0]:
# How does the data look now?
df.head()


In [0]:
# What about the tail?
df.tail()


In [0]:
# A time indexed data frame provides much more control over the data
df.loc[df.index > '2019-10-15']


In [0]:
# We can look at a single party
df.lpc.loc['2019-10-20']


In [0]:
# We can focus on a subset of columns
parties = ["lpc", "cpc", "ndp", "bq", "gpc"]
df.loc[:, parties]


In [0]:
# We can aggregate/resample the data
df[parties].resample('D', how='mean').head()


In [0]:
# We can also use pandas to plot
df[parties].resample('D', how='mean').plot()



### Anatomy of a Figure
![Anatomy of a Figure](https://matplotlib.org/3.1.1/_images/anatomy.png)


In [0]:
# We can look at the distributions for each party
df[parties].plot(kind='kde')


In [0]:
# Or do a simple box-plot
df[parties].boxplot()


In [0]:
# Let's look at missing values
df.isnull().mean()


In [0]:
# We can remove missing values
df.dropna()


In [0]:
# We just lost half of our dataset...
# Maybe we should fill the missing values
tmp_df = df.fillna(method='ffill', limit=3).copy()
tmp_df.isnull().mean()

df = tmp_df


In [0]:
# Let's investigate which polling firms have been most active
df.source.value_counts()


In [0]:
# Remove the firms that released less than 5 polls
tmp_mask = df.source.value_counts() >= 5
mask = tmp_mask.index[tmp_mask]

df = df[df.source.isin(mask)]


In [0]:
# Once again we could decide to visualize directly the result
df.source.value_counts().plot(kind='barh')


In [0]:
# Try to do grouped operations and see how did each of these firms portrayed the liberal party
df.groupby('source').lpc.describe().sort_values(by='mean')


In [0]:
# We can also look at the means for all the parties
df.groupby('source')[parties].mean().sort_values('lpc')


In [0]:
# We can also apply custom functions by groups
z_score = lambda x: (x-x.mean()) / x.std()
df.reset_index().groupby('source')[parties].apply(z_score).head()


In [0]:
# Most algorithms need you to shape the date in a long format
long_df = pd.melt(
    df.reset_index(),
    id_vars=['date', 'source'],
    value_vars=parties,
    var_name='party',
    value_name='share',
)

long_df.head()


In [0]:
# Seaborn, a statistical data visualization library uses long-format
import seaborn as sns
sns.set(style="whitegrid", palette="muted")

sns.swarmplot(
    x="party",
    y="share",
    hue="source",
    data=long_df,
)


In [0]:
# What if we need to add the sample size back?
new_df = long_df.merge(
    df[['method', 'source']].reset_index(),
    on=['date', 'source']
)

new_df.head()


In [0]:
# We can also expand the dataframe back to a wide format
new_df = new_df.pivot_table(
    index=['date', 'source', 'method'],
    columns='party',
    values='share',
)

new_df.head()

In [0]:
new_df.stack()